In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import torch
from transformers import LightOnOcrForConditionalGeneration, LightOnOcrProcessor

device = (
    "mps"
    if torch.backends.mps.is_available()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
dtype = torch.float32 if device == "mps" else torch.bfloat16

model = LightOnOcrForConditionalGeneration.from_pretrained(
    "lightonai/LightOnOCR-2-1B-bbox", torch_dtype=dtype
).to(device)
processor = LightOnOcrProcessor.from_pretrained("lightonai/LightOnOCR-2-1B-bbox")

url = "https://huggingface.co/datasets/hf-internal-testing/fixtures_ocr/resolve/main/SROIE-receipt.jpeg"

conversation = [{"role": "user", "content": [{"type": "image", "url": url}]}]

inputs = processor.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)
# Output will include bounding boxes like this(no space): ![image](image_1.png)120,50,850,400

You are using a model of type `mistral3` to instantiate a model of type `lighton_ocr`. This may be expected if you are loading a checkpoint that shares a subset of the architecture (e.g., loading a `sam2_video` checkpoint into `Sam2Model`), but is otherwise not supported and can yield errors. Please verify that the checkpoint is compatible with the model you are instantiating.


Loading weights:   0%|          | 0/532 [00:00<?, ?it/s]

In [5]:
inputs = {
    k: v.to(device=device, dtype=dtype) if v.is_floating_point() else v.to(device)
    for k, v in inputs.items()
}

output_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids = output_ids[0, inputs["input_ids"].shape[1] :]
output_text = processor.decode(generated_ids, skip_special_tokens=True)
print(output_text)

Document No : TD01167104

Date : 25/12/2018 8:13:39 PM

Cashier : MANIS

Member :

# CASH BILL

<table>
  <thead>
    <tr>
      <th>CODE/DESC</th>
      <th>PRICE</th>
      <th>Disc</th>
      <th>AMOUNT</th>
    </tr>
    <tr>
      <th>QTY</th>
      <th>RM</th>
      <th></th>
      <th>RM</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>9556939040118</td>
      <td>KF MODELLING CLAY KIDDY FISH</td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>1 PC *</td>
      <td>9.000</td>
      <td>0.00</td>
      <td>9.00</td>
    </tr>
  </tbody>
</table>

Total : 9.00

Rounding Adjustment : 0.00

**Rounded Total (RM): 9.00**


In [ ]:
from langchain_core.output_parsers import XMLOutputParser

output_parser = XMLOutputParser()

output_parser.parse(output_text)